# DSCI 512 Lecture 7: Discrete Optimization

Outline:

- Discrete optimization introduction 
- Constraints
- Example: TA assignment
- Formulating an optimization problem
- Types of discrete optimization problems
- T/F questions

## Discrete optimization introduction

- _Optimization_ refers to maximizing or minimizing a function with respect to its inputs.
  - For example, find the minimum of $f(x)=(x-0.9)^2$. The answer is $0$, when $x=0.9$.
- _Discrete optimization_ refers to optimization where the variables are _discrete_ (as opposed to continuous).
  - For example, find the minimum of $f(x)=(x-0.9)^2$ where $x$ is an integer. The answer is $0.01$, when $x=1$.




#### Example problem

Another example: find the length-8 string of letters (e.g. `aaaaaaaa`, `abcdefgh`, etc.) that contains the most English words in it.

- Because the problem is discrete, we can count the number of "options" or "configurations".
  - In the first case above, $x$ is any integer so there are infinitely many.
  - In this example, if we're just using the 26 lower-case letters, we have $26^8$ possibilities.

In [3]:
26**8

208827064576

- That is a lot!


If the number of possibilities is finite, you can enumerate (or list) all possible configurations (inputs).
  - In our example, there is `aaaaaaaa`, `aaaaaaab`, `aaaaaaac`, ..., `zzzzzzzz`. 
  - This is called "brute force search".
  - Often, it's not computationally feasible to search through all possible combinations because there are too many. 

#### Specifying an optimization problem

To specify the problem, we need to specify a few things:

1. A specification of the space of possible inputs.
2. An _objective function_ which takes an input and computes a score. 
3. Are we maximizing or minimizing?

In the above, the space of possible inputs is all length-8 strings. And the objective function is the number of English words in the string, and we are maximizing it. 

#### Solving an optimization problem with brute force (example)

- First, let's implement the objective function in Python

In [10]:
from nltk.corpus import words

word_list = {word for word in words.words() if word.islower() and len(word) > 1}

def english_words_in_string(s):
    """
    Find all the English words within a string.
    
    Parameters
    ----------
    s : str
        The input string.
       
    Returns
    -------
    set
        The set of English words contained in s.
        
    Example
    -------
    >>> english_words_in_string("theapple")
    {'a', 'apple', 'e', 'ea', 'h', 'he', 'heap', 'l', 'p', 't', 'th', 'the'}    
    """
    
    words = set()
    for start in range(len(s)):
        for end in range(start+1, len(s)+1):
            if s[start:end] in word_list:
                words.add(s[start:end])
    return words

english_words_in_string("theapple")

{'apple', 'ea', 'he', 'heap', 'th', 'the'}

It looks like the length-8 string `theapple` contains a few words (though is `th` really a word?! And what about the word `app`?!).

Next, let's implement the objective function:

In [11]:
def objective_fun(s):
    """Count the number of English words contained within a string."""
    return len(english_words_in_string(s))

objective_fun("theapple")

6

The objective function for `theapple` is 6. We are trying to find the string (or strings - the solution is not always unique) that make this value as large as possible.

Let's try this only using the first 5 letters of the alphabet, so the code isn't too slow. We should only have

In [3]:
5**8

390625

possibilities, which is reasonable. We'll grab some code from lab 2:

In [12]:
def get_inputs(letters, n):
    """
    Return all length-n strings using letters.
    
    Parameters
    ----------
    letters : list
        The list of letters (could be a string or a list)
    
    n : int
        The desired length.
        
    Returns
    -------
    list
        A list of the resulting strings.
        
    Examples
    --------
    get_inputs(['a', 'b', 'c'], 2)
    ['aa', 'ab', 'ac', 'ba', 'bb', 'bc', 'ca', 'cb', 'cc']
    """
    
    if n == 0:
        return [""]
    
    return [letter + l for letter in letters for l in get_inputs(letters, n-1)]

In [13]:
inputs = get_inputs(['a', 'b', 'c', 'd', 'e'], 8)

In [14]:
inputs[:10] # print out the first 10

['aaaaaaaa',
 'aaaaaaab',
 'aaaaaaac',
 'aaaaaaad',
 'aaaaaaae',
 'aaaaaaba',
 'aaaaaabb',
 'aaaaaabc',
 'aaaaaabd',
 'aaaaaabe']

In [15]:
import numpy as np

scores = list(map(objective_fun, inputs))
np.max(scores)

13

13 words, wow! Let's see what string led to this:

In [9]:
best_s = inputs[np.argmax(scores)]
best_s

'cabacade'

And the words:

In [10]:
english_words_in_string(best_s)

{'aba',
 'abac',
 'abaca',
 'ad',
 'ade',
 'ba',
 'bac',
 'ca',
 'cab',
 'caba',
 'cad',
 'cade',
 'de'}

We can also check if the solution is unique:

In [11]:
np.sort(scores)

array([ 0,  0,  0, ..., 13, 13, 13])

Looks like the solution is not unique (we see several 13's). 

#### Another example: shortest path in a graph

- Last week, we talked about finding the shortest path between two nodes $A$ and $B$ in a graph.
- This is itself a discrete optimization problem.

1. Possible inputs: all possible paths from $A$ to $B$.
2. Objective function: the length of the path.
3. We are minimizing.
4. We won't actually do this because there is an even better way, see next lecture

#### Solving an optimization problem

- There are many ways to go about solving an optimization problem. 
- In general, one possibility is _brute force_ in which all possible inputs are considered.
- That's the approach we just took in our toy example above.
- But this is often infeasible because it would be too slow.
  - Think about $26^8$ strings.
  - Think about all possible paths between two nodes in the Facebook graph!
- So, the hard part about optimization is often in finding clever ways to solve the problem faster, that still give the correct solution.
  - E.g. breadth-first search to find the shortest distance in a graph.
  - For the English words problem, there is nothing better than brute force. But there often is.
- The other hard part can be coverting a conceptual idea into the right mathematical form as an optimization problem. This is called _formulating_ the problem.

## Constraints

- A popular way of expressing the space of possible inputs is to put _constraints_ on the possibilities.
- Example: all length-8 strings such that each letter appears at most twice.
  - `aabbccdd` is valid, `abcdefgh` is valid, but `aaabbbcc` is _not_ valid ("violates the constraint").
- You could say that this is just a change in the possible inputs, and it is.
  - But, because of how these problems are solved under the hood, we often think of constraints as a separate thing.
  

(Updated) To specify an optimization problem, we need to specify a few things:

1. A specification of the space of possible inputs.
2. An _objective function_ which takes an input and computes a score. 
4. (Optional) A set of _constraints_, which take an input and return true/false.
3. Are we maximizing or minimizing?

In [12]:
def characters_appear_at_most_twice(s, n=2):
    """
    Checks whether all characters appear at most n times in s.
    If any character appears n+1 or more times, returns False.
    
    Parameters
    ----------
    s : str
        The input string
    n : int, optional
        The maximum number of times a character is allowed to appear (default: 2)
        
    Returns
    -------
    bool
        Whether or not all characters appear at most twice
        
    Examples
    --------
    >>> characters_appear_at_most_twice("aabbccdef")
    True
    >>> characters_appear_at_most_twice("aaa")
    False
    >>> characters_appear_at_most_twice("aaa", 5)
    True
    """
    
    return max(s.count(char) for char in s) <= n

In [13]:
assert characters_appear_at_most_twice("aabbccdef")
assert not characters_appear_at_most_twice("aaa")
assert characters_appear_at_most_twice("aaa", 5)

Now we can solve the _constrained optimization problem_:

In [14]:
inputs[:10] # the ones from before

['aaaaaaaa',
 'aaaaaaab',
 'aaaaaaac',
 'aaaaaaad',
 'aaaaaaae',
 'aaaaaaba',
 'aaaaaabb',
 'aaaaaabc',
 'aaaaaabd',
 'aaaaaabe']

In [15]:
constrained_inputs = [s for s in inputs if characters_appear_at_most_twice(s)]

In [16]:
constrained_inputs[:10]

['aabbccdd',
 'aabbccde',
 'aabbcced',
 'aabbccee',
 'aabbcdcd',
 'aabbcdce',
 'aabbcddc',
 'aabbcdde',
 'aabbcdec',
 'aabbcded']

In [17]:
constrained_scores = list(map(objective_fun, constrained_inputs))
np.max(constrained_scores)

12

In [18]:
constrained_best_s = constrained_inputs[np.argmax(constrained_scores)]
constrained_best_s

'babedade'

In [19]:
english_words_in_string(constrained_best_s)

{'abed',
 'ad',
 'ade',
 'ba',
 'babe',
 'be',
 'bed',
 'bedad',
 'da',
 'dad',
 'dade',
 'de'}

Some things to note:

- The constrained optimum can be equal to, or worse than, the unconstrained optimum.
  - In this case it was worse - we got 12 words instead of 13 by not allowing repeats. 
- It is possible to have multiple constraints.
  - In this case we only had one constraint.
  - In general, for an input to be valid, _all_ the constraints must be satisfied. 
  - For example: each letter appears at most twice AND the string is a palindrome AND it does not contain the substring "ab" (3 constraints).

## Example: TA assignment

Consider the problem of assigning TAs to labs in MDS.

In [2]:
import pandas as pd

TAs_df = pd.read_csv("data/TA_apps.csv").set_index("name")
str_to_list = lambda s: list(map(int,s.split(",")))
TAs_df["can_teach"   ] = TAs_df["can_teach"  ].apply(str_to_list)
TAs_df["enthusiastic"] = TAs_df["enthusiastic"].apply(str_to_list)
TAs_df

,availability,R_proficiency,python_proficiency,can_teach,enthusiastic
name,,,,,
Alice,MTW,competent,expert,"[542, 551, 552]","[511, 571, 513]"
Bob,MTWR,competent,none,"[551, 523, 531, 552, 561]","[551, 523, 561]"
Carol,MT,expert,expert,"[542, 551, 523, 552, 571, 513]","[551, 552, 571]"
Chuck,MW,expert,beginner,"[511, 551, 531, 561]","[521, 542, 523]"
Dan,MTR,expert,expert,"[511, 521, 512, 552, 532, 571, 513]","[521, 512, 552]"
Erin,R,expert,expert,"[512, 532, 513]","[512, 513]"
Faith,MTWR,competent,expert,"[511, 521, 523, 531, 512, 561, 571, 513]","[521, 512, 513]"
Grace,MR,beginner,expert,"[521, 523, 512]",[511]
Heidi,TWR,expert,competent,[531],"[511, 551, 523, 552, 561, 532, 571]"


In [3]:
TAs = TAs_df.index.values.tolist()

In [4]:
courses_df = pd.read_csv("data/MDS_courses_term_1.csv").set_index("course_number")
courses_df

,course_title,slug,block,primary_lang,lab_days
course_number,,,,,
511,Programming for Data Science,prog-dsci,1,both,WR
521,Computing Platforms for Data Science,platforms-dsci,1,both,MT
542,Communication and Argumentation,comm-arg,1,none,WR
551,Descriptive Statistics and Probability for Dat...,stat-prob-dsci,1,none,MT
512,Algorithms and Data Structures,alg-data-struct,2,python,WR
523,Data Wrangling,data-wrangling,2,R,WR
531,Data Visualization I,viz-1,2,both,MT
552,Statistical Inference and Computation I,stat-inf-1,2,R,MT
513,Databases and Data Retrieval,database-data-retr,3,both,WR


In [5]:
courses = courses_df.index.values.tolist()

In [6]:
blocks = set(courses_df["block"])

Goal: match TAs to courses so that our staffing needs are covered in the best way possible.

Ok, let's make this more concrete. Here are the **constraints**:

- Each course should be assigned exactly 2 TAs.
- A TA can only cover one course at a time (i.e., in a given block).
- A TA can only be assigned to a course they have listed as "can teach" or "enthusiastic to teach".
- To cover a course, the TA must be available for one of the two lab days (for simplicity).

And here is the **objective**:

- We want to maximize the number of assigned courses that TAs are enthusiastic about.

Let's try solving this by hand (on the whiteboard)

## Formulating an optimization problem using PuLP

We can solve discrete optimization problems like the TA assignment problem using the PuLP Linear Programming package. Let's work through the steps required to do this

In [7]:
import pulp

#### First, are we maximizing or minimizing?

We are maximizing. Let's set up our "problem"

In [8]:
prob = pulp.LpProblem("TA assignments", pulp.LpMaximize)

#### Setting up the decision variables

- For this problem, what are our _decision variables_? 
- Turns out such problems are best handled with _binary variables_.

Let $x_{ij}$ be $1$ if TA $i$ is assigned to course $j$, and 0 otherwise.

In [17]:
x_df = pd.DataFrame(data=(np.random.rand(len(TAs),len(courses))<.2).astype(int), index=TAs, columns=courses)
x_df

,511,521,542,551,512,523,531,552,513,561,571,532
Alice,0,0,0,0,0,0,0,0,0,0,0,0
Bob,1,0,0,0,1,1,0,0,0,0,0,1
Carol,0,0,0,1,0,0,1,1,0,1,0,0
Chuck,0,0,1,0,0,0,0,0,0,0,1,0
Dan,0,1,0,0,0,0,0,0,0,0,0,0
Erin,1,1,0,0,0,1,1,0,0,0,1,0
Faith,1,0,0,0,0,0,1,0,0,0,0,0
Grace,0,0,1,0,0,0,0,1,0,0,0,0
Heidi,1,1,0,0,0,0,0,0,0,0,0,1
Ivan,0,1,0,1,0,0,0,0,0,0,0,0


In [18]:
x = pulp.LpVariable.dicts("x", (TAs, courses), 0, 1, pulp.LpInteger) 

- (Optional) How many possibilities are there?
  - With $11$ TAs, $12$ courses, and $2$ TAs per course: around $11^{12\times 2}\approx10^{25}$
  - With $11\times 12$ binary variables, we get $2^{11\times12}\approx10^{40}$.
  - Our formulation has _more_ configurations because it doesn't have the "2 TAs per course" constraint hardwired into it. This seems "redundant" but is actually the way to go because it allows us to use PuLP.

#### Setting up the constraints

We now have to change our English constraints into constraints on $x$. IN PuLP, you just "add" the contraints to the problem. Your contraints should be a boolean expressions where at one side is some (pulp) function of the decision variable.

_Each course should be assigned exactly 2 TAs._
  - For all courses $j$, we require $\sum_i x_{ij} = 2$
  - That is, the sum of TAs for DSCI 511 equals 2, the sum of TAs for DSCI 521 equals 2, ...


In [19]:
TAS_PER_COURSE = 2

for course in courses:
    prob += pulp.lpSum(x[TA][course] for TA in TAs) == TAS_PER_COURSE

You don't has to use comprehensions here, but you do need to use lpSum, you cannot sum using sum or np.sum. The below is equivalent to the above.

In [20]:
for course in courses:
    course_TAs = []
    for TA in TAs:
        course_TAs.append(x[TA][course])
    prob += pulp.lpSum(course_TAs) == TAS_PER_COURSE

<br>

_A TA can only cover one course at a time (i.e., in a given block)._
  - For all TAs $i$, for all blocks, we require $\sum_j x_{ij} \leq 1$
  - That is, the sum of Alice's Block 1 courses is at most 1, the sum of Bob's Block 1 courses is at most 1, ...

In [21]:
# my code here
for TA in TAs:
    for block in blocks:
        prob += pulp.lpSum(x[TA][course] for course in courses if courses_df.loc[course]["block"] == block) <= 1
# my code here

<br> 

_A TA can only be assigned to a course they have listed as "can teach" or "enthusiastic to teach"._
  - For all $i,j$ such that canteach$(i,j)$ is false and enthusiastic$(i,j)$ is false, $x_{ij}=0$
  - That is, Bob cannot teach 511 because canteach(Bob,511) is false and enthusiastic(Bob,511) is false, ...
 


In [36]:
# my code here
for TA in TAs:
    for course in courses:
        if course not in TAs_df.loc[TA]["can_teach"] and course not in TAs_df.loc[TA]["enthusiastic"]:
            prob += x[TA][course] == 0
# my code here

<br> 

Exercise: _To cover a course, the TA must be available for one of the two lab days._
  - For all $i,j$ such that available$(i,\text{day1}(j))$ is false and available$(i,\text{day2}(j))$ is false, $x_{ij}=0$
  - That is, Carol cannot teach DSCI 542 because available(Carol,W) is false and available(Carol,Th) is false
    - And day1(542) is W, day2(542) is Th.
  

In [37]:
#your code here
for TA in TAs:
    for course in courses:
        if courses_df.loc[course]["lab_days"][0] not in TAs_df.loc[TA]["availability"] and courses_df.loc[course]["lab_days"][1] not in TAs_df.loc[TA]["availability"]:
            prob += x[TA][course] == 0
#your code here

#### Setting up the objective

_We want to maximize the number of assigned courses that TAs are enthusiastic about._
  - Maximize the following objective: for all $i,j$ such that enthusiastic$(i,j)$ is true, $\sum_{ij} x_{ij}$
  
(Optional) Another way of writing this:
  - Maximize $\sum_{ij} \text{enthusiastic}(i,j)x_{ij}$
  - This assumes the values of enthusiastic$(i,j)$ are either 0 or 1, so that the product 1 is only when the TA is enthusiastic and assigned to the course.
  - It's generally a common pattern to be multiplying some known quantities by the decision variables.
  
In PuLP you also just "add" the objective function. PuLP figures out the difference between the constraints and the objective function based on their type (the former is boolean)

In [38]:
prob += pulp.lpSum(x[TA][course] for TA in TAs for course in courses if course in TAs_df.loc[TA]["enthusiastic"])

#### Solving the problem

In [39]:
prob.solve()

1

In [40]:
pulp.LpStatus[prob.status]

'Optimal'

In [41]:
print("We have %d enthusiastic courses out of a possible %d." % 
      (pulp.value(prob.objective), len(courses)*TAS_PER_COURSE))

We have 21 enthusiastic courses out of a possible 24.


#### Printing the results

In [42]:
out_df_by_TA = pd.DataFrame("", index=TAs, columns=blocks)
for TA in TAs:
    for course in courses:
        if pulp.value(x[TA][course]) == 1:
            out_df_by_TA.at[TA, courses_df.loc[course]["block"]] = course
out_df_by_TA

,1,2,3
Alice,,,571
Bob,551,531,561
Carol,551,552,571
Chuck,542,523,561
Dan,521,552,
Erin,,512,513
Faith,521,512,513
Grace,511,,
Heidi,,523,532
Ivan,511,,532


In [43]:
for course in courses:
    print(course, end=": ")
    for TA in TAs:
        if pulp.value(x[TA][course]) == 1:
            print(TA, end=', ')
    print("")

511: Grace, Ivan, 
521: Dan, Faith, 
542: Chuck, Judy, 
551: Bob, Carol, 
512: Erin, Faith, 
523: Chuck, Heidi, 
531: Bob, Judy, 
552: Carol, Dan, 
513: Erin, Faith, 
561: Bob, Chuck, 
571: Alice, Carol, 
532: Heidi, Ivan, 


In [44]:
# Enthusiastic courses
for course in courses:
    for TA in TAs:
        if pulp.value(x[TA][course]) == 1 and course in TAs_df.loc[TA]["enthusiastic"]:
            print("%-7s is enthusiastic about DSCI %d!" % (TA, course))

Grace   is enthusiastic about DSCI 511!
Ivan    is enthusiastic about DSCI 511!
Dan     is enthusiastic about DSCI 521!
Faith   is enthusiastic about DSCI 521!
Chuck   is enthusiastic about DSCI 542!
Judy    is enthusiastic about DSCI 542!
Bob     is enthusiastic about DSCI 551!
Carol   is enthusiastic about DSCI 551!
Erin    is enthusiastic about DSCI 512!
Faith   is enthusiastic about DSCI 512!
Chuck   is enthusiastic about DSCI 523!
Heidi   is enthusiastic about DSCI 523!
Judy    is enthusiastic about DSCI 531!
Carol   is enthusiastic about DSCI 552!
Dan     is enthusiastic about DSCI 552!
Erin    is enthusiastic about DSCI 513!
Faith   is enthusiastic about DSCI 513!
Bob     is enthusiastic about DSCI 561!
Alice   is enthusiastic about DSCI 571!
Carol   is enthusiastic about DSCI 571!
Heidi   is enthusiastic about DSCI 532!


#### What they actually do for MDS TAs:

- Match TA programming language proficiency with courses.
- Respect TA preferences on max or min number of courses (e.g. 1-2 courses, exactly 3 courses, etc).
- Allow the script to under/over staff if necessary (big term in the objective).
- And more!


## Types of discrete optimization problems

- We tend to classify discrete optimization problems into different types, based on the tricks we can use to solve them.
- For example, some tough optimization problems are said to be [NP-complete](https://en.wikipedia.org/wiki/NP-completeness). We won't go into detail here, but it's related to the idea that you have to use brute force.
- Other optimization problems are called _linear_. Discrete optimization problems that are linear are called [integer linear programs](https://en.wikipedia.org/wiki/Integer_programming). 
  - There is a technical definition of linear optimization problems, but in short the objective and constraints must be linear functions of the inputs. 
  - In Python, we can conveniently solve linear optimization problems using the [PuLP](https://pythonhosted.org/PuLP/) package.
  - The way these solvers work is outside the scope of MDS.
- In general, if your optimization problem falls into a more restrictive class of problems, there are probably tools to solve them faster. 
- Next class we'll talk about another class of problems that can be solved fast, using "dynamic programming".

## T/F questions

1. A solver will violate the constraints if this results in huge gains for the objective function.
2. All discrete optimization problems have unique solution.
3. All discrete optimization problems have a solution.
4. PuLP can be use to solve any optimization problem, but it may be very slow for some problems.